In [1]:
import requests
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

In [2]:
stats=pd.read_excel('statid.xlsx')

In [65]:
games=[]
for j in range(1,18):
    print(j)
    url='http://www.nfl.com/ajax/scorestrip?season={}&seasonType={}&week={}'.format(2018,"REG",j)



    root=ET.fromstring(requests.get(url).text)


    for i in root[0]:
        games.append(i.attrib['eid'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [62]:
url

'http://www.nfl.com/ajax/scorestrip?season=2018&seasonType=REG&week=3'

In [66]:
games

['2018090600',
 '2018090900',
 '2018090901',
 '2018090902',
 '2018090903',
 '2018090904',
 '2018090905',
 '2018090906',
 '2018090907',
 '2018090908',
 '2018090909',
 '2018090910',
 '2018090911',
 '2018090912',
 '2018091000',
 '2018091001',
 '2018091300',
 '2018091600',
 '2018091601',
 '2018091602',
 '2018091603',
 '2018091604',
 '2018091605',
 '2018091606',
 '2018091607',
 '2018091608',
 '2018091609',
 '2018091610',
 '2018091611',
 '2018091612',
 '2018091613',
 '2018091700',
 '2018092000',
 '2018092300',
 '2018092301',
 '2018092302',
 '2018092303',
 '2018092304',
 '2018092305',
 '2018092306',
 '2018092307',
 '2018092308',
 '2018092309',
 '2018092310',
 '2018092311',
 '2018092312',
 '2018092313',
 '2018092400',
 '2018092700',
 '2018093005',
 '2018093006',
 '2018093007',
 '2018093000',
 '2018093001',
 '2018093002',
 '2018093003',
 '2018093004',
 '2018093008',
 '2018093009',
 '2018093010',
 '2018093011',
 '2018093012',
 '2018100100',
 '2018100400',
 '2018100700',
 '2018100701',
 '20181007

In [7]:
gid=2019080860
# for gid in games:
url2='http://www.nfl.com/liveupdate/game-center/{}/{}_gtd.json'.format(gid,gid)

d=requests.get(url2).json()

dfs=[]
for key,val in d[str(gid)]['drives'].items():
    if key =='crntdrv':
        continue
    df=pd.DataFrame(val)[['start','end']].dropna()

    df2=pd.DataFrame([0])
    counter=0
    for i in df.columns:
        counter=0
        for j in df[i]:
            df2[i+' '+df[i].index[counter]]=j
            counter+=1
    df2=df2.drop([0,'start team','end team','end qtr'],axis=1)



    df=pd.DataFrame(val).drop(['plays','start','end'],axis=1).drop_duplicates()
    try:
        df=df.drop('redzone',axis=1)
    except:
        pass

    df=df.reset_index().drop('index',axis=1).reset_index()

    df2=df2.reset_index()

    df=pd.merge(df,df2,on='index').drop('index',axis=1)
    df['driveid']=key
    dfs.append(df)

    df=pd.concat(dfs).fillna(0)

    df['posteam']=df['posteam'].str.pad(3,'right')

    df['end yrdln']=np.where(df['end yrdln']=='',df['start yrdln'],df['end yrdln'])

    df['start yrdln']=np.where(df['posteam']==df['start yrdln'].str[:3],
                               df['start yrdln'].str.split(' ').str[1],
                               (50-df['start yrdln'].str.split(' ').str[1].astype(np.int64))+50)

    df['end yrdln']=df['start yrdln'].astype(np.int64)+df['penyds'].astype(np.int64)+df['ydsgained'].astype(np.int64)
    df['gid']=gid

df

,posteam,qtr,fds,result,penyds,ydsgained,numplays,postime,start qtr,start time,start yrdln,end time,end yrdln,driveid,gid
0,NYJ,1,3,Touchdown,0,75,10,3:07,1,15:00,25,11:53,100,1,2019080860
0,NYG,1,0,Punt,0,4,5,1:50,1,11:53,25,10:03,29,2,2019080860
0,NYJ,1,0,Punt,0,4,4,2:04,1,10:03,22,07:59,26,3,2019080860
0,NYG,1,5,Touchdown,0,79,9,4:23,1,07:59,21,03:36,100,4,2019080860
0,NYJ,1,0,Punt,0,4,5,2:13,1,03:36,16,01:23,20,5,2019080860
0,NYG,2,3,Touchdown,0,81,9,2:42,1,01:23,19,13:41,100,6,2019080860
0,NYJ,2,4,Touchdown,26,49,14,6:17,2,13:41,25,07:24,100,7,2019080860
0,NYG,2,0,Punt,-15,10,7,2:11,2,07:24,34,05:13,29,8,2019080860
0,NYJ,2,1,Punt,-15,22,7,2:30,2,05:13,43,02:43,50,9,2019080860
0,NYG,2,0,Fumble,0,3,1,0:09,2,02:43,13,02:34,16,10,2019080860


In [76]:
gid=2018120900
dfs=[]
for gid in games:
    print (gid)
    url2='http://www.nfl.com/liveupdate/game-center/{}/{}_gtd.json'.format(gid,gid)

    d=requests.get(url2).json()


    for key,val in d[str(gid)]['drives'].items():
        if key =='crntdrv':
            continue

        for key2,val2 in val['plays'].items():
            for key3,val3 in val2['players'].items():
                df=(pd.DataFrame(val3))
                df['playerid']=key3
                df['play']=key2
                df['drive']=key
                df['description']=val2['desc']
                df['gid']=gid
                df['posteam']=val2['posteam']
                dfs.append(df)
df=pd.concat(dfs)
# print (gid)
# url2='http://www.nfl.com/liveupdate/game-center/{}/{}_gtd.json'.format(gid,gid)

# d=requests.get(url2).json()


# for key,val in d[str(gid)]['drives'].items():
#     if key =='crntdrv':
#         continue

#     for key2,val2 in val['plays'].items():
#         for key3,val3 in val2['players'].items():
#             df=(pd.DataFrame(val3))
#             df['playerid']=key3
#             df['play']=key2
#             df['drive']=key
#             df['description']=val2['desc']
#             df['posteam']=val2['posteam']
#             dfs.append(df)
# df=pd.concat(dfs)

2018090600
2018090900
2018090901
2018090902
2018090903
2018090904
2018090905
2018090906
2018090907
2018090908
2018090909
2018090910
2018090911
2018090912
2018091000
2018091001
2018091300
2018091600
2018091601
2018091602
2018091603
2018091604
2018091605
2018091606
2018091607
2018091608
2018091609
2018091610
2018091611
2018091612
2018091613
2018091700
2018092000
2018092300
2018092301
2018092302
2018092303
2018092304
2018092305
2018092306
2018092307
2018092308
2018092309
2018092310
2018092311
2018092312
2018092313
2018092400
2018092700
2018093005
2018093006
2018093007
2018093000
2018093001
2018093002
2018093003
2018093004
2018093008
2018093009
2018093010
2018093011
2018093012
2018100100
2018100400
2018100700
2018100701
2018100702
2018100703
2018100704
2018100705
2018100706
2018100707
2018100708
2018100709
2018100710
2018100711
2018100712
2018100800
2018101100
2018101400
2018101401
2018101402
2018101403
2018101404
2018101405
2018101406
2018101407
2018101408
2018101409
2018101410
2018101411

In [77]:
url2

'http://www.nfl.com/liveupdate/game-center/2018123011/2018123011_gtd.json'

In [78]:
# df=pd.merge(df,stats,left_on='statId',right_on='id',how='left')

# df['play']=df['play'].astype(np.int64)

# df['points']=df['yds points'].fillna(0)*df['yards'].fillna(0)+df['existence points'].fillna(0)

# df['playerName'] = np.where(df['cat']=='defense', df['clubcode'],df['playerName'])

# piv=df.pivot_table(index='playerName',values=['points','yards'],aggfunc=sum)

# df.to_clipboard(sep='\t')

In [79]:
import statid

In [80]:
statdfs=[]
for key,val in statid.idmap.items():
    tdf=pd.DataFrame(val)
    tdf['statId']=key
    statdfs.append(tdf)

In [81]:
st=pd.concat(statdfs)

In [82]:
df2=pd.merge(df,st,on='statId',how='left')

In [83]:
df2

,clubcode,playerName,sequence,statId,yards,playerid,play,drive,description,gid,posteam,cat,fields,desc,long
0,PHI,J.Elliott,1,410,66.0,00-0033787,37,1,J.Elliott kicks 65 yards from PHI 35 to end zo...,2018090600,PHI,kicking,kicking_all_yds,Kickoff and length of kick,Kickoff and length of kick. Includes end zone ...
1,PHI,J.Elliott,2,44,65.0,00-0033787,37,1,J.Elliott kicks 65 yards from PHI 35 to end zo...,2018090600,PHI,kicking,kicking_tot,Kickoff with touchback,Kickoff resulted in a touchback.
2,PHI,J.Elliott,2,44,65.0,00-0033787,37,1,J.Elliott kicks 65 yards from PHI 35 to end zo...,2018090600,PHI,kicking,kicking_touchback,Kickoff with touchback,Kickoff resulted in a touchback.
3,PHI,J.Elliott,2,44,65.0,00-0033787,37,1,J.Elliott kicks 65 yards from PHI 35 to end zo...,2018090600,PHI,kicking,kicking_yds,Kickoff with touchback,Kickoff resulted in a touchback.
4,ATL,None,3,51,0.0,0,37,1,J.Elliott kicks 65 yards from PHI 35 to end zo...,2018090600,PHI,team,kickret_touchback,Kickoff - touchback,Kick resulted in a touchback. A touchback impl...
5,ATL,L.Paulsen,1,93,5.0,00-0027215,52,1,"(15:00) PENALTY on ATL-L.Paulsen, False Start,...",2018090600,ATL,penalty,penalty,Penalty,
6,ATL,L.Paulsen,1,93,5.0,00-0027215,52,1,"(15:00) PENALTY on ATL-L.Paulsen, False Start,...",2018090600,ATL,penalty,penalty_yds,Penalty,
7,ATL,M.Ryan,1,15,10.0,00-0026143,75,1,(15:00) M.Ryan pass short right to J.Jones pus...,2018090600,ATL,passing,passing_att,Passing yards,Passing yards and a pass attempt completed.
8,ATL,M.Ryan,1,15,10.0,00-0026143,75,1,(15:00) M.Ryan pass short right to J.Jones pus...,2018090600,ATL,passing,passing_cmp,Passing yards,Passing yards and a pass attempt completed.
9,ATL,M.Ryan,1,15,10.0,00-0026143,75,1,(15:00) M.Ryan pass short right to J.Jones pus...,2018090600,ATL,passing,passing_yds,Passing yards,Passing yards and a pass attempt completed.


In [84]:
df3=pd.merge(df2,pd.read_excel('points.xlsx'),
         left_on=['cat','fields'],
         right_on=['cat2','field']).\
drop({'cat_x','fields','cat2'},axis=1).\
    rename({'cat_y':'cat'},axis=1)

In [85]:
df3['points']=np.where(df3['cat']=='yds',df3['points']*df3['yards'],df3['points'])

In [86]:
df3['playerName']=np.where(df3['clubcode']==df3['posteam'],df3['playerName'],df3['clubcode'])

In [87]:
df3.to_csv('2018 Season.csv',index=False)